# $\bm{21}$ $\enspace$ **多输入多输出通道**

当我们添加通道时，我们的输入和隐藏的表示都变成了三维张量。例如，每个 RGB 输入图像具有 $3\times h\times w$ 的形状。我们将这个大小为 $3$ 的轴称为通道 (channel) 维度。

## $\bm{21.1}$ $\enspace$ **多输入通道**

![两个输入通道的互相关计算](image/两个输入通道的互相关计算.jpg)

In [1]:
import torch
from d2l import torch as d2l


def corr2d_multi_in(X, K):
    # 先遍历 "X" 和 "K" 的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))


接下来我们验证互相关的输出。

In [2]:
X = torch.tensor(
    [
        [[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
        [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]],
    ]
)
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

## $\bm{21.2}$ $\enspace$ **多输出通道**

用 $c_i$ 和 $c_o$ 分别表示输入和输出通道的数目，并让 $k_h$ 和 $k_w$ 为卷积核的高度和宽度。为了获得多个通道的输出，我们可以为每个输出通道创建一个形状为 $c_i\times k_h\times k_w$ 的卷积核张量，这样卷积核的形状是 $c_o\times c_i\times k_h\times k_w$。

In [3]:
def corr2d_multi_in_out(X, K):
    # 迭代 "K" 的第0个维度，每次都对输入 "X" 执行互相关运算
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

通过将核张量 $K$ 与 $K+1$（$K$ 中每个元素加 $1$）和 $K+2$ 连接起来，构造了一个具有 $3$ 个输出通道的卷积核。

In [4]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

我们对输入张量 $X$ 与卷积核张量 $K$ 执行互相关运算。现在的输出包含 $3$ 个通道，第一个通道的结果与先前输入张量 $X$ 和多输入单输出通道的结果一致。

In [5]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## $\bm{21.3}$ $\enspace$ **$1\times 1$ 卷积层**

下图展示了使用 $1\times 1$ 卷积核与 $3$ 个输入通道和 $2$ 个输出通道的互相关计算。这里的输入和输出具有相同的高度和宽度，输出的每个元素都是从输入图像中的同一位置的元素的线性组合。我们可以将 $1\times 1$ 卷积层看作在每个像素位置应用的全连接层，以 $c_i$ 个输入值转换为 $c_o$ 个输出值。因为这仍然是一个卷积层，所以跨像素的权重是一致的。同时，$1\times 1$ 卷积层需要的权重维度为 $c_o\times c_i$，再额外加上一个偏置。

![1 $\times$ 1卷积核](image/1x1卷积核.jpg)

In [6]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [7]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6